In [1]:
import imp
import pinatrace_dataset

import os
import torch
import torch.nn as nn
from mapping import Mapping
from utils import to_tensor_batched
import tqdm

/var/folders/wv/wmn__gfj1j188w6fb4gdzvn80000gn/T/ipykernel_7748/1582355159.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
from hwi_dataset import get_test_set, read_trace, get_reads, get_hwi_page_requests

page_size = 16*1024
test_dir_name = '11'
test_set = get_test_set()
requests = []

reads = []
for volume_paths in sorted(test_set[test_dir_name]):
    print(f"Volume {volume_paths[0].split('/')[-2]}")

    for path in tqdm.tqdm(volume_paths):
        reads += get_reads(read_trace(path)).tolist()

reads = sorted(reads, key=lambda x: x[-1])

requests = get_hwi_page_requests(reads, page_size=page_size)
requests = requests[:50_000]
print("unique pages", len(set(requests)))
print("length", len(requests))

Volume cache_trace_909


100%|██████████| 8/8 [00:01<00:00,  7.00it/s]


Volume cache_trace_916


100%|██████████| 9/9 [00:01<00:00,  7.48it/s]


Volume cache_trace_933


100%|██████████| 8/8 [00:01<00:00,  7.55it/s]

unique pages 10951
length 50000


In [6]:
print(requests)

[10816148, 10816404, 5671060, 5671316, 5287316, 2362772, 2363028, 7604628, 11111316, 11111572, 11111316, 11111572, 3846548, 3846804, 2511764, 2512020, 10444180, 10444436, 10444180, 10444436, 384404, 384660, 5420436, 5420692, 384404, 384660, 7865236, 7865492, 572564, 2526100, 2526356, 11692948, 11693204, 876692, 876948, 626068, 11644308, 11644564, 11644308, 11644564, 11744660, 11470228, 11470484, 3682964, 3683220, 7677332, 4238996, 4239252, 11756948, 11757204, 5350036, 1141396, 1141652, 8592532, 8592788, 546452, 546452, 546708, 546964, 547220, 547476, 547732, 547988, 548244, 548500, 548756, 549012, 549268, 549524, 549780, 550036, 550292, 550548, 9885332, 9885588, 2282132, 891540, 10652308, 10652564, 2443924, 2444180, 3966356, 3966612, 4579476, 12076436, 9866388, 12101780, 12102036, 1262996, 1263252, 9420948, 2120340, 2120596, 3608212, 3608212, 4597652, 12111508, 12111764, 5564564, 5564820, 928148, 928404, 9600148, 9600404, 10889364, 12320660, 12320916, 9665172, 9665428, 2087316, 7050644

In [14]:
from gc import enable
from math import e
from re import A
from httpx import request
from matplotlib.pylab import f
from numpy import add
from torch import threshold
import xgboost as xgb
from mapping import Mapping
from collections import deque, defaultdict
import pandas as pd
import tqdm
from sklearn.metrics import root_mean_squared_error, mean_squared_error
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, eta=0.1, enable_categorical=True)


deltas = defaultdict(lambda: deque(maxlen=32))
last_request = {}

train_samples = []
trace = pinatrace_dataset.read_trace()
read_trace = [x[2] for x in trace if x[1] == 'R']
CACHE_SIZE = 16
ADDRESS_POOL = 100000
DELTA_POOL = 50
TRAIN_SAMPLES = 64

train_tagged = {}

K = 5
L = 10

misses = 0
address_mapping = Mapping(max_size=ADDRESS_POOL, hash_mapping=True)
delta_mapping = Mapping(max_size=DELTA_POOL, hash_mapping=False)

threshold = 150
threshold_delta = 1e-3
is_fitted = False
EDC_SIZE = 10

decay_tasks = defaultdict(deque)

class EDC:

    def __init__(self, size=EDC_SIZE):
        self.size = size
        self.counters = [0] * size
        self.requests_count = 0
        self.last_t = 0

    def request(self, t):
        for i in range(self.size):
            self.counters[i] += 1
            def decay_task():
                self.counters[i] /= 2

            decay_tasks[t+(i+1)**2].append(decay_task)

    def get_dict(self):
        return {
            f'edc_{i}': self.counters[i] for i in range(self.size)
        }


class HeuristicCache:

    def __init__(self, max_size):
        self.max_size = max_size
        self.cache = deque(maxlen=max_size)

    def __contains__(self, address):
        return address in self.cache

    def __len__(self):
        return len(self.cache)

    def remove_from_tail(self):
        address = self.cache.pop()
        address_mapped = address_mapping.map(address)
        train_tagged[address_mapped] = True

        return address

    def insert(self, x, rank=None):
        self.cache.append(x)

    def remove(self, x):
        self.cache.remove(x)


cache = HeuristicCache(CACHE_SIZE)
edcs = defaultdict(EDC)


def get_model_input(address):
    address_mapped = address_mapping.map(address)
    address_deltas = [
        delta_mapping.map(x) + 1 for x in deltas[address_mapped]
    ]
    if len(address_deltas) < 32:
        address_deltas += [0] * (32 - len(address_deltas))

    deltas_dict = {
        f'delta_{i}': address_deltas[i] for i in range(32)
    }

    edcs_input = edcs[address_mapped].get_dict()

    dict_input = edcs_input | deltas_dict

    return dict_input


def mat_evict(t):
    global threshold
    global is_fitted

    for r in range(1, L):
        obj = cache.remove_from_tail()

        obj_mapped = address_mapping.map(obj)

        if not is_fitted:
            break

        m_input = get_model_input(obj)
        m_input = pd.DataFrame(m_input, index=[0])
        for i in range(32):
            m_input[f'delta_{i}'] = pd.Categorical(m_input[f'delta_{i}'], categories=range(DELTA_POOL+1))

        m_input = pd.get_dummies(m_input)

        model_tta = model.predict(m_input)[0]

        time_since_last_request = t - last_request[obj_mapped]

        if model_tta > time_since_last_request:
            tta = model_tta - time_since_last_request
        else:
            tta = time_since_last_request - model_tta

        if tta > threshold:
            break

        if r == L - 1:
            break

        cache.insert(obj)

    if not is_fitted:
        return

    if r > K:
        threshold = threshold * (1 - threshold_delta)

    if r < K:
        threshold = threshold * (1 + threshold_delta)

    return obj


def add_to_training_set(address):
    address_mapped = address_mapping.map(address)

    model_input = get_model_input(address)

    true_tta = deltas[address_mapped][-1]

    train_samples.append((model_input, true_tta))

    return train_samples

def train():
    global is_fitted

    input_df = pd.DataFrame([x[0] for x in train_samples])

    for i in range(32):
        input_df[f'delta_{i}'] = pd.Categorical(input_df[f'delta_{i}'], categories=range(DELTA_POOL+1))

    input_df = pd.get_dummies(input_df)

    output_df = pd.DataFrame([x[1] for x in train_samples], columns=['tta'])

    if is_fitted:
        model_out = model.predict(input_df)
        rmse = root_mean_squared_error(output_df, model_out)
    else:
        rmse = 0

    model.fit(input_df, output_df)

    score = model.score(input_df, output_df)

    is_fitted = True

    return rmse

n_scores = deque(maxlen=20)
pbar = tqdm.tqdm(enumerate(requests), total=len(requests))
score = 0
for t, address in pbar:

    while decay_tasks[t]:
        decay_tasks[t].popleft()()

    assert len(decay_tasks[t]) == 0
    del decay_tasks[t]

    address_mapped = address_mapping.map(address)
    if address_mapped in last_request:
        deltas[address_mapped].append(t - last_request[address_mapped])

    last_request[address_mapped] = t

    edcs[address_mapped].request(t)


    if train_tagged.get(address_mapped):
        add_to_training_set(address)
        train_tagged[address_mapped] = False

    if address in cache:
        cache.remove(address)
        cache.insert(address)
    else:
        misses += 1
        if len(cache) == CACHE_SIZE:
            mat_evict(t)

        assert len(cache) < CACHE_SIZE
        cache.insert(address)

    if len(train_samples) == TRAIN_SAMPLES:
        score = train()
        train_samples = []
        #train_tagged = {}

        n_scores.append(score)

    scores_avg = sum(n_scores) / len(n_scores) if n_scores else 0

    pbar.set_description(
        f"Score: {score:.2f}, "
        f"Scores_avg: {scores_avg:.2f}, "
        f"Threshold: {threshold:.2f}, "
        f"Misses_ratio: {misses/(t+1):.3f}, "
        f"is_fitted: {is_fitted}, "
        #f"tagged to train: {sum(train_tagged.values())}, "
        f"train smpl: {len(train_samples)}"
    )


Score: 5170.47, Scores_avg: 4727.54, Threshold: 6709.45, Misses_ratio: 0.984, is_fitted: True, train smpl: 21:  47%|████▋     | 23652/50000 [19:32:19<28:40:48,  3.92s/it]   